In [1]:
import numpy as np
import pandas as pd
import itertools
# import matplotlib.pyplot as plt

from datetime import timedelta
from time import time

import pickle
import os

from dask.distributed import Client, progress
import dask.bag as db

# import edslab_prophet_dask as edd
# import edslab_prophet as ed

pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_rows', 100)


from fbprophet import Prophet

In [2]:
client = Client(processes = True,n_workers=12)
client

Client Scheduler: tcp://127.0.0.1:56712 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 12 Cores: 12 Memory: 34.14 GB


In [2]:
path = 'D:/BUDS_MID/OJT ATM/ATM Forecasting Data New/'
HP = pd.read_csv(path + 'Actual/Holidays_ALL.csv')

In [4]:
old_rmse = pd.read_csv('D:/BUDS_MID/OJT ATM/ATM Forecasting Data/Prophet best model.csv', usecols=['atm','model','rmse'])
old_rmse.columns=['tid','old_model','old_rmse']
old_rmse.tid = old_rmse.tid.astype('str')

In [5]:
old_rmse

,tid,old_model,old_rmse
0,91018579,HP32,45327.06
1,91018853,HP05,71029.42
2,91017756,HP03,78171.41
3,91017824,HP03,83593.64
4,BR324,HP36,89855.17
...,...,...,...
459,BR163,HP32,1151344.66
460,BR996,HP36,1162954.27
461,BR129,HP06,1196989.52
462,BR981,HP36,1211367.02


In [23]:
rmse = pd.concat([pd.read_csv(path+'/Prophet Train/'+file, usecols=['tid','model','dev_rmse','val_rmse'],
                             dtype={'tid':'str'}) for file in os.listdir(path+'/Prophet Train/')],ignore_index=True)
rmse['mean_rmse'] = rmse[['dev_rmse','val_rmse']].mean(axis=1) 
rmse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8262 entries, 0 to 8261
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tid        8262 non-null   object 
 1   model      8262 non-null   object 
 2   dev_rmse   8262 non-null   float64
 3   val_rmse   8262 non-null   float64
 4   mean_rmse  8262 non-null   float64
dtypes: float64(3), object(2)
memory usage: 322.9+ KB


In [24]:
# rmse.merge(old_rmse,how='left').to_csv(path+'/83 Train RMSE and old.csv')

In [25]:
best = rmse.groupby(['tid']).val_rmse.apply(pd.Series.argmin).map(rmse[:18].model.to_dict()).reset_index()

In [26]:
best

,tid,val_rmse
0,91010033,HP36
1,91010053,HP36
2,91010068,HP32
3,91010075,HP36
4,91010090,HP36
...,...,...
454,BR398,HP30
455,BR400,HP33
456,BR404,HP33
457,BR405,HP34


In [27]:
best.columns=['tid','best_model']
best.tid = best.tid.astype(str)
best['week'] = best.best_model.str[2:3].astype(int)
best['mon'] = best.best_model.str[3:].astype(int)
best.best_model.value_counts()

HP32    81
HP30    63
HP34    56
HP36    35
HP20    32
HP33    31
HP22    29
HP35    26
HP00    22
HP24    18
HP23    15
HP02    11
HP25    11
HP03     9
HP26     7
HP04     5
HP05     4
HP06     4
Name: best_model, dtype: int64

In [28]:
best.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459 entries, 0 to 458
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tid         459 non-null    object
 1   best_model  459 non-null    object
 2   week        459 non-null    int32 
 3   mon         459 non-null    int32 
dtypes: int32(2), object(2)
memory usage: 10.9+ KB


In [29]:
rmse.head()

,tid,model,dev_rmse,val_rmse,mean_rmse
0,91010033,HP00,298556.65,236843.07,267699.86
1,91010033,HP02,286657.34,223539.27,255098.31
2,91010033,HP03,285876.87,227278.71,256577.79
3,91010033,HP04,282278.96,224213.21,253246.09
4,91010033,HP05,281348.43,224981.58,253165.00


In [31]:
best = best.merge(rmse, left_on=['tid','best_model'], right_on=['tid','model'])

In [32]:
best.to_csv(path+'89 Best Model.csv')

In [21]:
atms = best.tid.unique()
len(atms)

459

In [12]:
data = pd.read_pickle(path+'Actual/82 Aggregated TIDBR WDL.pkl')
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 751436 entries, 0 to 751435
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   DATE         751436 non-null  datetime64[ns]
 1   TID_BR       751436 non-null  object        
 2   WDL_AMT      751436 non-null  float64       
 3   WDL_AMT_FIN  751436 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 22.9+ MB


In [14]:
data_train = data[data.DATE <= '2019-12-31'][['DATE','TID_BR','WDL_AMT_FIN']].copy()
data_train.columns=['ds','tid','y']
data_train

,ds,tid,y
0,2017-01-01,91010033,0.00
1,2017-01-01,91010053,0.00
2,2017-01-01,91010068,0.00
3,2017-01-01,91010075,0.00
4,2017-01-01,91010090,0.00
...,...,...,...
720505,2019-12-31,BR987,0.00
720506,2019-12-31,BR988,1247500.00
720507,2019-12-31,BR992,2717100.00
720508,2019-12-31,BR996,4859000.00


In [15]:
week = [0,2,3]
mon = [0,2,3,4,5,6]
hol = ['HP']
holidays=HP

scenario =  pd.DataFrame(list(itertools.product(hol,week,mon)),columns=['hol','week','mon'])
scenario['model'] = scenario.hol + scenario.week.astype('str') +scenario.mon.astype(str)
scenario['hol_num'] = scenario['hol'].map({'HP':0,'OP':1})

print('\ntotal scenario: {}'.format(scenario.shape[0]))


total scenario: 18


In [16]:
dev_length = sum(data_train.ds.dt.date.unique() < pd.to_datetime('20181231'))
print(dev_length)
period = '1 days'
horizon = 14

729


In [17]:
from fbprophet import Prophet

In [18]:
def train(series,val_dates,holidays,weekly=0,monthly=0):
    series = series.copy()
    val_dates = val_dates.copy()
    #define model
    modelf = Prophet(interval_width = 0.95,
                    daily_seasonality = False,
                    weekly_seasonality = False,
                    yearly_seasonality = False,
                    holidays = holidays)
    if weekly != 0: modelf.add_seasonality(name='weeklyx', period = 7, fourier_order = weekly)
    if monthly != 0: modelf.add_seasonality(name='monthlyx', period = 30.5, fourier_order = monthly)
    
    modelf.fit(series)
    
#     return modelf
    val_dates['yhat'] = modelf.predict(val_dates[['ds']]).yhat.values.clip(min=0)
    val_dates['pred_date'] = series.tail(1).ds.dt.date.values.tolist()[0]
    val_dates['pred_day'] = ['day '+str(i) for i in range(val_dates.shape[0])]
    return val_dates

In [22]:
for atm in atms:
    start=time()

    series = data_train[data_train.tid == atm][['ds','y']].copy().reset_index(drop=True)
    index = best[best.tid == atm].index
    week = best.week[index].tolist()[0]
    mon = best.mon[index].tolist()[0]
    n=series.shape[0]-dev_length-horizon-1

    dbseries = db.from_sequence([series[i:i+dev_length] for i in range(n)])
    dbforecast = db.from_sequence([series[i+dev_length:i+dev_length+horizon][['ds','y']] for i in range(n)])

    dbmaster = db.map(train,dbseries,dbforecast,HP,week,mon)


    model = dbmaster.compute()
    model=pd.concat([model[i] for i in range(len(model))])
    model['se'] = (model.y-model.yhat)**2
    model['tid'] = atm
    rmse = model.groupby([model.ds.dt.month,'pred_day']).se.mean().agg(np.sqrt).reset_index()
    model.to_csv(path+'Prophet Forecast/'+atm+'_forecast.csv')
    rmse.to_csv(path+'Prophet Forecast RMSE/'+atm+'_forecast_rmse.csv')
    print(f'\nATM:{atm}  RMSE:{rmse.se.mean()}')
    print(timedelta(seconds=time()-start))


ATM:91010033  RMSE:262511.6321450931
0:02:55.407647

ATM:91010053  RMSE:283324.79593418085
0:02:54.044770

ATM:91010068  RMSE:71257.62660345524
0:02:50.002248

ATM:91010075  RMSE:210095.41561052905
0:03:21.350033

ATM:91010090  RMSE:467238.9709510498
0:02:55.833219

ATM:91010101  RMSE:362764.56462696515
0:02:55.567786

ATM:91010125  RMSE:746794.2127539003
0:03:03.546278

ATM:91010126  RMSE:403909.5648522778
0:02:58.224047

ATM:91010127  RMSE:309767.1422313532
0:03:01.533490

ATM:91010133  RMSE:515152.47271006653
0:03:01.369259

ATM:91010135  RMSE:528150.7415995076
0:03:02.133590

ATM:91010145  RMSE:113521.83386170492
0:03:03.074064

ATM:91010151  RMSE:370725.20234062936
0:02:55.631876

ATM:91010152  RMSE:543385.2994815469
0:02:54.177683

ATM:91010155  RMSE:460845.6256721458
0:03:02.230544

ATM:91010156  RMSE:544226.4453197134
0:02:57.906718

ATM:91010157  RMSE:662196.631606108
0:02:51.167067

ATM:91010158  RMSE:315715.81825257663
0:02:55.869826

ATM:91010162  RMSE:414326.8884058954
0:

KeyboardInterrupt: 

### Get best model